In [1]:
import requests
import fake_useragent
from typing import List

## Загрузка данных

In [2]:
def get_all_regions() -> List[str]:
    user_agent = fake_useragent.UserAgent()
    regions = requests.get(
        url="https://api.hh.ru/areas",
        headers={"user_agent": user_agent.random}
    ).json()
    return [str(region["id"]) for region in regions[0]["areas"]]

In [21]:
def get_count_vacancies(job_title: str, regions: List[str]) -> dict[str, dict[str, int]]:
    user_agent = fake_useragent.UserAgent()
    mean_salary_by_region = []
    result_row = {}
    for region in get_all_regions():
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&per_page=100&area={region}"
        vacancies = requests.get(
            url=url,
            headers={"user_agent": user_agent.random}                    
        ).json()
        for vacancy in vacancies["items"]:
            if vacancy["salary"]["from"] is None:
                continue
            mean_salary_by_region.append(vacancy["salary"]["from"])
    result_row[job_title] = {
        "Средняя зарплата.": int(sum(mean_salary_by_region) / len(mean_salary_by_region)),
        "Количество вакансий в России.": len(mean_salary_by_region) * 17.8125
    }
    return result_row

In [24]:
jt = "продавец"
get_count_vacancies(job_title=jt, regions=get_all_regions())

{'продавец': {'Средняя зарплата.': 39927,
  'Количество вакансий в России.': 129897}}

In [23]:
user_agent = fake_useragent.UserAgent()
data = requests.get(
    url="https://omsk.rabota.ru/vacancy/?query=java&sort=relevance&all_regions=1",
    headers={"user_agent": user_agent.random} 
)